In [1]:
!pip install pandas

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
!pip install --upgrade pip

    100% |████████████████████████████████| 1.3MB 3.1MB/s ta 0:00:01
distributed 1.21.8 requires msgpack, which is not installed.
  Found existing installation: pip 10.0.1
    Uninstalling pip-10.0.1:
      Successfully uninstalled pip-10.0.1


In [5]:
!pip install msgpack

    100% |████████████████████████████████| 317kB 4.5MB/s ta 0:00:01


In [55]:
!pip install xgboost

    100% |████████████████████████████████| 15.8MB 607kB/s ta 0:00:01  5% |█▊                              | 829kB 1.0MB/s eta 0:00:15    32% |██████████▎                     | 5.1MB 2.0MB/s eta 0:00:06    51% |████████████████▍               | 8.1MB 1.7MB/s eta 0:00:05    58% |██████████████████▉             | 9.3MB 2.1MB/s eta 0:00:04    98% |███████████████████████████████▌| 15.6MB 3.4MB/s eta 0:00:01    99% |████████████████████████████████| 15.8MB 4.0MB/s eta 0:00:01


In [57]:
!pip install hyperopt

    100% |████████████████████████████████| 122kB 181kB/s ta 0:00:01
    100% |████████████████████████████████| 829kB 605kB/s ta 0:00:01
    100% |████████████████████████████████| 409kB 324kB/s ta 0:00:011
  Running setup.py bdist_wheel for future ... done
  Stored in directory: /home/bumblebee/.cache/pip/wheels/bf/c9/a3/c538d90ef17cf7823fa51fc701a7a7a910a80f6a405bf15b1a
Successfully built future


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from xgboost.sklearn import XGBRegressor
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
import math
from sklearn.metrics import mean_squared_error
import time

In [2]:
%matplotlib inline

In [3]:
!dir

Beer_hackathon.ipynb  Beer\ Test\ Data\ Set.csv  Beer\ Train\ Data\ Set.csv


In [3]:
train_Data = pd.read_csv("Beer Train Data Set.csv")

In [5]:
print(train_Data.columns.values)

['ABV' 'Brewing Company' 'Food Paring' 'Glassware Used' 'Beer Name'
 'Ratings' 'Style Name' 'Cellar Temperature' 'Serving Temperature' 'Score']


In [4]:
space = {
        'n_estimators': hp.uniform('n_estimators', 100, 1000),
        'eta': hp.quniform('eta', 0.025, 0.5, 0.025),
        # A problem with max_depth casted to float instead of int with
        # the hp.quniform method.
        'max_depth':  hp.choice('max_depth', np.arange(1, 60, dtype=int)),
        'min_child_weight': hp.quniform('min_child_weight', 1, 6, 1),
        'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
        'gamma': hp.quniform('gamma', 0.5, 1, 0.05),
        'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
        # Increase this number if you have more cores. Otherwise, remove it and it will default
        # to the maxium number.
        'nthread': 4,
        "xg_reg_alpha": hp.loguniform('xg_reg_alpha',1e-9,1.0),
        "xg_reg_lambda":hp.loguniform('xg_reg_lambda',1e-9,1)

    }

In [7]:
print(len(train_Data["Style Name"].unique()))

94


Find a way to encode beer style name.

In [8]:
train_Data["Style Name"].value_counts()

AmericanIPA                      25577
AmericanPaleAle(APA)             12648
AmericanPorter                   11992
AmericanDoubleImperialIPA        10147
SaisonFarmhouseAle                8982
AmericanAmberRedAle               5924
AmericanWildAle                   5523
AmericanDoubleImperialStout       5125
AmericanStout                     4911
AmericanBlondeAle                 4234
FruitVegetableBeer                4088
AmericanBrownAle                  3958
Hefeweizen                        3670
Witbier                           3265
GermanPilsener                    2955
BelgianPaleAle                    2909
EuroPaleLager                     2444
BerlinerWeissbier                 2376
AmericanBlackAle                  2255
MrzenOktoberfest                  2214
EnglishPaleAle                    2176
EnglishBitter                     2164
AmericanPaleLager                 2160
Tripel                            1987
Klsch                             1888
CzechPilsener            

In [14]:
train_Data.isna().sum()

ABV                    15130
Brewing Company            0
Food Paring                0
Glassware Used             0
Beer Name                  0
Ratings                    0
Style Name                 0
Cellar Temperature      6781
Serving Temperature      193
Score                      0
dtype: int64

In [15]:
food_pairing_unique_list = (train_Data["Food Paring"].unique())

In [11]:
print(train_Data["Brewing Company"].max())

13541


In [5]:
train_Data = shuffle(train_Data, random_state=2)

In [6]:
train_Data.reset_index(drop=True,inplace=True)

In [ ]:
#train_Data.drop(labels=["Cellar Temperature"],inplace=True,axis=1)

In [7]:
#handle ABV
mean_abv = train_Data["ABV"].mean()

print(mean_abv)

train_Data["ABV"].fillna(mean_abv, inplace=True)

6.354960736131313


In [29]:
(train_Data["Cellar Temperature"].value_counts().index)[0]

'40-45'

In [8]:
train_Data["Cellar Temperature"].fillna((train_Data["Cellar Temperature"].value_counts().index)[0],inplace=True)

In [9]:
#converting into float from string
train_Data["Ratings"] = train_Data['Ratings'].str.replace(',', '')
train_Data["Ratings"] = train_Data['Ratings'].astype(np.float64)

In [10]:
3427086540917546
validation_rmse 0.3601428124655341 = pd.get_dummies(train_Data["Cellar Temperature"])

In [11]:
cellar_temp_dataframe.reset_index(drop=True,inplace=True)

In [12]:

print(cellar_temp_dataframe.head(n=5))

   35-40  40-45  45-50
0      1      0      0
1      1      0      0
2      0      0      1
3      0      0      1
4      0      1      0


In [25]:
#as of now not taking serving temperature as a input
#train_Data_Wo_Serving_Null = train_Data[train_Data["Serving Temperature"].notnull()]

In [14]:
train_Data.isna().sum()

ABV                      0
Brewing Company          0
Food Paring              0
Glassware Used           0
Beer Name                0
Ratings                  0
Style Name               0
Cellar Temperature       0
Serving Temperature    193
Score                    0
dtype: int64

In [16]:
#handling style input
unique_beer_style_names = train_Data["Style Name"].unique()

In [17]:
dict_style_name_value = {}

for style_name in unique_beer_style_names:
    Score_with_style_name = train_Data[train_Data["Style Name"] == style_name]["Score"]
    dict_style_name_value[style_name] = Score_with_style_name.sum() / len(Score_with_style_name)

train_Data["Style Name"].replace(to_replace=dict_style_name_value,inplace=True)
    
    

In [20]:
#handling style input
food_pairing_unique_list = train_Data["Food Paring"].unique()

In [21]:
dict_food_pairing_value = {}

for food_pairing in food_pairing_unique_list:
    Score_with_food_pairing = train_Data[train_Data["Food Paring"] == food_pairing]["Score"]
    dict_food_pairing_value[food_pairing] = Score_with_food_pairing.sum() / len(Score_with_food_pairing)

train_Data["Food Paring"].replace(to_replace=dict_food_pairing_value,inplace=True)


/home/bumblebee/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [22]:
print(train_Data.isna().sum())

ABV                      0
Brewing Company          0
Food Paring              0
Glassware Used           0
Beer Name                0
Ratings                  0
Style Name               0
Cellar Temperature       0
Serving Temperature    193
Score                    0
dtype: int64


In [23]:
final_df = pd.concat([train_Data,cellar_temp_dataframe],axis=1)

In [24]:
print(final_df.isna().sum())

ABV                      0
Brewing Company          0
Food Paring              0
Glassware Used           0
Beer Name                0
Ratings                  0
Style Name               0
Cellar Temperature       0
Serving Temperature    193
Score                    0
35-40                    0
40-45                    0
45-50                    0
dtype: int64


In [36]:
print(final_df.dtypes)

ABV                    float64
Brewing Company        float64
Food Paring            float64
Glassware Used          object
Beer Name              float64
Ratings                float64
Style Name             float64
Serving Temperature     object
Score                  float64
35-40                    uint8
40-45                    uint8
45-50                    uint8
dtype: object


In [27]:
final_df.rename(columns={"35-40": "c_t1", "40-45": "c_t2","45-50":"c_t3"},inplace=True)
print(final_df[["Food Paring","Style Name","c_t1", "c_t2",
 "c_t3","ABV","Ratings"]].head(n=10))


   Food Paring  Style Name  c_t1  c_t2  c_t3        ABV  Ratings
0     3.174928    3.174928     1     0     0   6.000000      4.0
1     2.965526    2.965526     1     0     0   5.800000      3.0
2     3.304736    3.304736     0     0     1  10.000000      1.0
3     3.674808    3.674808     0     0     1   6.000000   1016.0
4     3.256399    3.256399     0     1     0   8.500000     33.0
5     2.990949    2.990949     0     1     0   6.354961      1.0
6     3.495354    3.495354     0     1     0   6.354961      1.0
7     3.160955    3.160955     0     0     1   6.700000     13.0
8     3.174928    3.174928     1     0     0   6.200000      0.0
9     3.067656    3.067656     0     1     0   5.500000      3.0


In [28]:
columns_to_train = ["ABV","Ratings","Style Name","Food Paring","c_t1", "c_t2",
 "c_t3"]
X_train, X_validtn, y_train, y_validtn = train_test_split(final_df[columns_to_train], final_df["Score"], test_size=0.2,
                                                          random_state=1)


In [29]:
print(X_train.isna().sum())

ABV            0
Ratings        0
Style Name     0
Food Paring    0
c_t1           0
c_t2           0
c_t3           0
dtype: int64


In [41]:
print(y_train.head(n=5))

110550    3.89
23028     4.32
38859     3.60
120251    2.61
177166    3.44
Name: Score, dtype: float64


In [27]:
#319,0.021239072071744414,32,3,5.318135711773575e-05,5.318135711773575e-05,0.05e-08,0.08177818529253764,1.738085684913476e-08trials = Trials()

In [ ]:

xg_model = XGBRegressor(n_estimators=319, learning_rate=0.021239072071744414, n_jobs=-1,
                            max_depth=32, gamma=3, colsample_bytree=5.318135711773575e-05,
                            min_child_weight=0.05e-08, reg_alpha=0.08177818529253764, subsample=params['subsample'],
                            reg_lambda=1.738085684913476e-08
                            )

In [ ]:
xg_model.fit(X_train, y_train)

In [32]:
trials = Trials()

In [35]:
def train_xg_boost(params):

    
    xg_model = XGBRegressor(n_estimators=int(params['n_estimators']), learning_rate=params['eta'], n_jobs=-1,
                            max_depth=int(params['max_depth']), gamma=params['gamma'], colsample_bytree=params['colsample_bytree'],
                            min_child_weight=params['min_child_weight'], reg_alpha=params['xg_reg_alpha'], subsample=params['subsample'],
                            reg_lambda=params['xg_reg_lambda']
                            )
    
    
    print("hyperparameters",params)
    # In[ ]:

    xg_model.fit(X_train.values, y_train.values)

    # In[68]:

    training_values = xg_model.predict(X_train.values)
    print(training_values)
        
    training_rmse = math.sqrt(mean_squared_error(y_train, training_values))
    
    print("training_rmse", training_rmse)
    validation_values = xg_model.predict(X_validtn.values)
    validation_rmse = math.sqrt(mean_squared_error(y_validtn, validation_values))
    print("validation_rmse", validation_rmse)

    return {'loss': validation_rmse,
        'status': STATUS_OK,
        'eval_time': time.time(),
        }





In [36]:
best = fmin(fn=train_xg_boost, space=space, algo=tpe.suggest,max_evals=100,trials=trials)



hyperparameters {'colsample_bytree': 0.8, 'eta': 0.35000000000000003, 'gamma': 0.6000000000000001, 'max_depth': 57, 'min_child_weight': 5.0, 'n_estimators': 530.8943989377739, 'nthread': 4, 'subsample': 0.9500000000000001, 'xg_reg_alpha': 1.2276574602966575, 'xg_reg_lambda': 1.010845164207668}
[3.5522807 3.7869232 3.7831292 ... 3.6578543 3.8998313 3.9655118]
training_rmse 0.34309129227386054
validation_rmse 0.3596886548326828
hyperparameters {'colsample_bytree': 0.75, 'eta': 0.42500000000000004, 'gamma': 0.6000000000000001, 'max_depth': 39, 'min_child_weight': 2.0, 'n_estimators': 880.4294537153836, 'nthread': 4, 'subsample': 0.7000000000000001, 'xg_reg_alpha': 1.4533793705880464, 'xg_reg_lambda': 1.4370619648453016}
[3.619413  3.8002303 3.7970445 ... 3.6592023 3.9305935 3.9065287]
training_rmse 0.3419906582341722
validation_rmse 0.3616829109254794
hyperparameters {'colsample_bytree': 0.9500000000000001, 'eta': 0.325, 'gamma': 0.8, 'max_depth': 8, 'min_child_weight': 6.0, 'n_estimators

KeyboardInterrupt: 